Define a tiny example (4 frames) with reduced distributions:
- dirichlet over operations: $[1,1,1,2,1]$
- hmm: 3 states, 2 units 
- gmm: 1 1d component



In [6]:
import numpy as np
import amdtk
import subprocess




alphas = np.array([1,1,1,2,1])

mus = np.array([-2, 2])
sigmas = np.array([1,1])

t = np.array([[.5, .5, 0,0,0,0],
 [0,.5,.5,0,0,0,0],
 [0,0,.5,.5,0,0,0],
 [0,0,0,.5,.5,0,0],
 [.5,0,0,0,0,0,.5]])

data = np.array([-3,-1,2,3])
tops = np.array([0,1])


state_llh = []


In [5]:

model = amdtk.PhoneLoopNoisyChannel.create(
    n_units=2,  # number of acoustic units
    n_states=3,   # number of states per unit
    n_comp_per_state=1,   # number of Gaussians per emission
    n_top_units=2, # size of top PLU alphabet
    mean=np.zeros_like(mus), 
    var=np.ones_like(sigmas),
    max_slip_factor=.05
    #concentration=conc
)

model.generate_start_items(tops, )






[-2.08333333 -2.08333333 -2.08333333 -2.08333333 -2.08333333]
[-2.08333333 -2.08333333 -2.08333333 -2.08333333 -2.08333333]


AttributeError: 'PhoneLoopNoisyChannel' object has no attribute 'mean'